In [1]:
import os 
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


## 1. RunnableParallel

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 创建vector store
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
# 创建检索器
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    # 与RunnableParallel({"context": retriever, "question": RunaablePassthrough()})等价
    # question根据用户输入获得
    # retriever根据question获取context
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

## 2. itemgetter

In [4]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        # 用户输入question和language
        # itemgetter获取question
        # question传入retriever，得到的返回值赋给context
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

## 3. Steps Parallel

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

# 定义joke_chain和poem_chain
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

# 把两个chain并行执行，并返回一个字典
# 字典的键是创建RunnableParallel时自定的参数
# 在本例中是 "joke" 和 "poem" 
# 与{"joke": joke_chain, "poem": poem_chain}等效
# 与 RunnableParallel({"joke": joke_chain, "poem": poem_chain})等效
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

# 并行可以加快速度
map_chain.invoke({"topic": "bear"})///

{'joke': AIMessage(content='Why did the bear bring a ladder to the bar? \nBecause he heard the drinks were on the house!'),
 'poem': AIMessage(content='In the dark of night, the bear roams free\nA wild soul, fierce and full of mystery')}